In [1]:
!nvidia-smi

Thu Mar 30 02:50:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/BTCV

/content/drive/MyDrive/BTCV


In [4]:
!pwd

/content/drive/MyDrive/BTCV


In [5]:
!python -c "import monai; import nibabel; import tqdm" || pip install -q "monai-weekly[nibabel, tqdm]"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.6 MB/s eta 0:00:00


In [6]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.8/308.8 KB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [8]:
import json
import os
import re

json_path = os.path.join("./dataset/dataset0", "dataset_0.json")
with open(json_path) as f:
    data = json.load(f)

train_path = os.path.join("Abdomen", "RegData", "Training-Training", "img")
test_path = os.path.join("Abdomen", "RegData", "Training-Testing", "img")

train_list = []
valid_list = []
for dir in os.listdir(os.path.join("./dataset", train_path)):
    for img in os.listdir(os.path.join("./dataset", train_path, dir)):
        if int(re.findall(r'\d+', img)[0]) < 35:
            train_list.append(os.path.join(train_path, dir, img))
        else:
            valid_list.append(os.path.join(train_path, dir, img))

test_list = []
for dir in os.listdir(os.path.join("./dataset", test_path)):
    for img in os.listdir(os.path.join("./dataset", test_path, dir)):
        test_list.append(os.path.join(test_path, dir, img))

data["test"] = []
data["training"] = []
data["validation"] = []

for img in train_list:
    data["training"].append({"image":img, "label":img.replace("img", "label")})
for img in valid_list:
    data["validation"].append({"image":img, "label":img.replace("img", "label")})
for img in test_list:
    data["test"].append({"image":img, "label":img.replace("img", "label")})

json_path = os.path.join("./dataset/", "dataset.json")
with open(json_path, "w") as f:
    json.dump(data, f)

In [ ]:
import torch
from medclip import MedCLIPModel, MedCLIPVisionModelViT, MedCLIPProcessor
import nibabel as nib
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

organs = {1: "spleen",
          2: "right kidney",
          3: "left kidney",
          4: "gallbladder",
          5: "esophagus",
          6: "liver",
          7: "stomach",
          8: "aorta",
          9: "inferior vena cava",
          10: "portal vein and splenic vein",
          11: "pancreas",
          12: "right adrenal gland",
          13: "left adrenal gland"}

processor = MedCLIPProcessor()
medclip_model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
medclip_model.from_pretrained()
medclip_model.to(device)

def to_medclip(label):
    label_unique = torch.unique(label)
    text = []
    for label in label_unique:
        if label != 0:
            text.append(organs[int(label)])
    text = ", ".join(text)
    inputs = processor(text=[text], return_tensors="pt", padding=True)
    text_embeds = medclip_model.encode_text(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    #text_embeds = torch.rand(1, 512)
    return text_embeds

train_path = os.path.join("Abdomen", "RegData", "Training-Training", "img")
test_path = os.path.join("Abdomen", "RegData", "Training-Testing", "img")

for dir in os.listdir(os.path.join("./dataset", train_path)):
    dir_path = os.path.join("./dataset", train_path.replace("img", "text"), dir)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    for img in os.listdir(os.path.join("./dataset", train_path, dir)):
        nii_label = nib.load(os.path.join("./dataset", train_path.replace("img", "label"), dir, img.replace("img", "label")))
        label_data = torch.tensor(nii_label.get_fdata())
        text_embeds = to_medclip(label_data)
        text_path = os.path.join("./dataset", train_path.replace("img", "text"), dir, img.replace("img", "text").replace("nii.gz", "pt"))
        torch.save(text_embeds, text_path)

for dir in os.listdir(os.path.join("./dataset", test_path)):
    dir_path = os.path.join("./dataset", test_path.replace("img", "text"), dir)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    for img in os.listdir(os.path.join("./dataset", test_path, dir)):
        nii_label = nib.load(os.path.join("./dataset", test_path.replace("img", "label"), dir, img.replace("img", "label")))
        label_data = torch.tensor(nii_label.get_fdata())
        text_embeds = to_medclip(label_data)
        text_path = os.path.join("./dataset", test_path.replace("img", "text"), dir, img.replace("img", "text").replace("nii.gz", "pt"))
        torch.save(text_embeds, text_path)

Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transfo

load model weight from: ./pretrained/medclip-vit


In [14]:
import torch
from medclip import MedCLIPModel, MedCLIPVisionModelViT, MedCLIPProcessor
import numpy as np
import itertools
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

organs = {1: "spleen",
          2: "right kidney",
          3: "left kidney",
          4: "gallbladder",
          5: "esophagus",
          6: "liver",
          7: "stomach",
          8: "aorta",
          9: "inferior vena cava",
          10: "portal vein and splenic vein",
          11: "pancreas",
          12: "right adrenal gland",
          13: "left adrenal gland"}

processor = MedCLIPProcessor()
medclip_model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
medclip_model.from_pretrained()
medclip_model.to(device)

text_path = os.path.join("./dataset", "Abdomen", "text")

def to_medclip(text):
    inputs = processor(text=[text], return_tensors="pt", padding=True)
    text_embeds = medclip_model.encode_text(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    return text_embeds

classes = range(1, 14)
combinations = []
for i in range(0, 14):
    combinations += list(itertools.combinations(classes, i))

for comb in combinations:
    text = []
    text_file = []
    for x in classes:
        if x in comb:
            text_file.append(str(x))
            text.append(organs[x])
    text_file = "_".join(text_file)
    text = " , ".join(text)
    if len(text) == 0:
        text = "none"
    text_embeds = to_medclip(text)
    if len(text_file) == 0:
        text_file = "0"
    torch.save(text_embeds, os.path.join(text_path, text_file + ".pt"))

Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'c

load model weight from: ./pretrained/medclip-vit


In [21]:
!python main.py --resume_ckpt --use_checkpoint --save_checkpoint --max_epochs 10 --val_every 10 --batch_size=2

2023-03-29 06:37:39.231524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 06:37:39.231668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 06:37:39.231690: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/local/lib/python3.9/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current defaul

In [22]:
!python main.py --attention --resume_ckpt --use_checkpoint --save_checkpoint --max_epochs 10 --val_every 10 --batch_size=2

2023-03-29 08:38:24.392256: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 08:38:24.392510: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 08:38:24.392538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/local/lib/python3.9/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current defaul

In [23]:
!python main.py --contrast --resume_ckpt --use_checkpoint --save_checkpoint --max_epochs 10 --val_every 10 --batch_size=2

2023-03-29 10:28:01.413718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 10:28:01.413878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 10:28:01.413904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/local/lib/python3.9/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current defaul

In [24]:
!python main.py --attention --contrast --resume_ckpt --use_checkpoint --save_checkpoint --max_epochs 10 --val_every 10 --batch_size=2

2023-03-29 12:18:12.934186: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 12:18:12.934374: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 12:18:12.934396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/local/lib/python3.9/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current defaul

In [7]:
!python main.py --attention --contrast --concat --resume_ckpt --use_checkpoint --save_checkpoint --max_epochs 10 --val_every 10 --batch_size=2

2023-03-29 13:38:15.174866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 13:38:15.336535: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-29 13:38:16.136178: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-29 13:38:16.136362: W tensorflow/compiler/xla/stream_executor

In [13]:
!python test.py

2023-03-30 03:06:16.062858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 03:06:16.230755: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-30 03:06:17.078538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-30 03:06:17.078681: W tensorflow/compiler/xla/stream_executor